In [1]:
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableSequence

c:\Users\Win 11\Desktop\RAG Pipeline\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1: Load and embed the document
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

llm=init_chat_model(model="openai:o4-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000261917D0FE0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026190F29310>, root_client=<openai.OpenAI object at 0x000002618EADCE60>, root_async_client=<openai.AsyncOpenAI object at 0x00000261917154C0>, model_name='o4-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [4]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
    You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

    Question: \"{question}\"

    Sub-questions:
    """)
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [6]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question=decomposition_chain.invoke({"question": query})
print(decomposition_question)

Sub-questions:

1. What memory architectures and storage mechanisms does LangChain employ in its workflows?  
2. How does LangChain define, orchestrate and execute its agents?  
3. What memory approaches and data-persistence features does CrewAI provide?  
4. How are agents designed, managed and invoked within CrewAI?


In [7]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
    Use the context below to answer the question.

    Context:
    {context}

    Question: {input}
    """)

qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [9]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]

    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input": subq, "context": docs})
        results.append(f"Q: {subq}\nA: {result}")

    return "\n\n".join(results)

In [10]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\\n")
print(final_answer)

✅ Final Answer:\n
Q: Here are four focused sub-questions you can use to guide your document search:
A: 1. Which external tools and environments does LangChain support connecting to (e.g., web search, calculators, code execution environments, custom APIs)?  
2. What vector databases does LangChain integrate with out-of-the-box (e.g., FAISS, Chroma, Pinecone, Weaviate)?  
3. How does LangChain enable semantic search over large document collections?  
4. In the context of Retrieval-Augmented Generation (RAG), how is external knowledge fetched from these sources and injected into the language model?

Q: What memory abstractions and storage options does LangChain provide, and how are they configured?
A: LangChain’s “memory” subsystem is deliberately factored into two orthogonal pieces:  
  1. A memory abstraction (the interface – what gets stored and how you read it back)  
  2. One or more concrete storage backends  

You pick a memory class (or compose several) to define *what* you want t